In [ ]:
import requests
import json
from langchain.chains import load_summarize_chain, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
# Initialize ChatOllama
ollama_api_key = "http://localhost:11434/v1/chat/completions"
llama_model = OllamaLLM(model="llama3.2:latest", api_key=ollama_api_key)

In [13]:
# Function 1: Load document using unstructuredFileLoader
def load_document(file_path):
    loader = UnstructuredFileLoader(file_path)
    document = loader.load()
    return document

In [14]:
# Function 2: Split documents into chunks using RecursiveCharacterTextSplitter
def split_document(document):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_documents(document)
    return chunks 

In [15]:
# Function 3: Summarize document using load_summarize_chain
def summarize_document(chunks):
    summarize_chain = load_summarize_chain(OllamaLLM(model="llama3.2:latest", api_key=ollama_api_key))
    summary = summarize_chain.run(chunks)
    return summary

In [16]:
# Function 4: Create QA chain
def create_qa_chain():
    qa_chain = load_qa_chain(OllamaLLM(model="llama3.2:latest", api_key=ollama_api_key), chain_type="map_reduce")
    return qa_chain

In [17]:
prompt_template = ChatPromptTemplate.from_template(
    "Based only on the following document summary, answer questions strictly within its context. "
    "If a question is outside the scope, respond with: "
    "‘This information is not in the document context, but I can provide general knowledge if you like.’\n\n"
    "Document Summary: {document_summary}\n\nUser Question: {question}"
)

In [18]:
# Function 5: Ask question from user and get an answer in the context of the document
def ask_question(qa_chain, question, document_summary):

    formatted_prompt = prompt_template.format(document_summary=document_summary, question=question)

    answer = qa_chain.run(input_documents=document_summary, question=question)
    return answer

In [19]:
# Function to define the interactive chatbot
def interactive_chatbot(qa_chain, document_summary):
    print("Welcome to the BZINT document Chatbot! Type 'exit' to stop.")

    conversation_history = []

    while True:
        # Get user input from the prompt
        user_input = input("You: ")

        # Exit condition
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        # Add user input to conversation history
        conversation_history.append(f"You: {user_input}")

        # Create a prompt string from the conversation history and document summary
        prompt = f"Document Summary: {document_summary}\n\n"  # Include the document summary at the start
        prompt += "\n".join(conversation_history)  # Append the conversation history

        # Send the prompt to the Ollama model for a response
        response = llama_model.invoke(prompt)

        # Add AI response to conversation history
        conversation_history.append(f"AI: {response}")

        print(f"You: {user_input}\nAI: {response}")

In [22]:
# Main function
def main():
    # Path to the document to be uploaded
    file_path = "C:\\Users\\nanadhirah\\Downloads\\SENIORSIGHT_THESIS_NUR AIN NADHIRAH ZAWAYI_2021829416_organized.pdf" 
    
    # Load the document
    document = load_document(file_path)

    # Split document into chunks
    chunks = split_document(document)

    # Summarize document
    summary = summarize_document(chunks)
    print("Summary of Document: ")
    print(summary)

    # Create QA chain
    qa_chain = create_qa_chain()

    # Ask a question (optional, just to check QA)
    #question = "Summarize this document"
    #answer = summarize_document(chunks)
    #print(f"Answer to the Question: {answer}")

    # Start the interactive chatbot session with document context
    interactive_chatbot(qa_chain, summary)

if __name__ == "__main__":
    main()

Summary of Document: 
Here is a concise summary:

The SeniorSight project developed an interactive dashboard to visualize trends in Malaysia's ageing population and support systems. The dashboard was successfully completed, delivering a comprehensive tool for stakeholders to make informed decisions and improve planning for the elderly. Key strengths include user-friendly design, robust data integration, and accessibility enhancements. Future recommendations include integrating real-time data feeds, enhancing predictive analytics, and collaborative partnerships with healthcare providers, NGOs, and academic institutions.
Welcome to the BZINT document Chatbot! Type 'exit' to stop.
You: who is this dashboard for?
AI: The SeniorSight project's interactive dashboard appears to be designed for stakeholders related to the elderly population in Malaysia, including:

1. Government officials involved in planning and policy-making.
2. Healthcare professionals working with older adults.
3. Non-gove